<a href="https://colab.research.google.com/github/profteachkids/CHE3023/blob/main/Example1_18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default='plotly_dark'

In [ ]:
D12=72e-6  # m2/s
D13=230e-6
D23 = 230e-6
P=101325  #Pa
T=548 #K
R=8.314 #J/(mol K)
c=P/(R*T)
kr=10  # mol/(m2 s mol frac)
delta=1e-3
F12=c*D12/delta
F13=c*D13/delta
F23=c*D23/delta
y1guess=0.1 # mol fraction at catalyst surface, initial guess
y2guess=0.5


In [ ]:
def f(yi):

    y1i=yi[0] # iterated mole fractions at catalyst surface
    y2i=yi[1]
    N1 = -kr*y1i
    N2=-N1
    N3=-N1
    def dy(eta, y):
        y1=y[0]
        y2=y[1]
        y3=(1-y1-y2)
        return np.array([(y1*N2-y2*N1)/F12 + (y1*N3-y3*N1)/F13,
                    (y2*N1-y1*N2)/F12+ (y2*N3- y3*N2)/F23])


    ybulk=solve_ivp(dy,(0,1),(y1i, y2i),method='Radau',dense_output=True).sol(1)
    return [ybulk[0]-0.6, ybulk[1]-0.2] 


In [ ]:
res=root(f, [y1guess, y2guess])
y1i=res.x[0]
y2i=res.x[1]
N1 = -kr*y1i
N2=-N1
N3=-N1
def dy(eta, y):
    y1=y[0]
    y2=y[1]
    y3=(1-y1-y2)
    return np.array([(y1*N2-y2*N1)/F12 + (y1*N3-y3*N1)/F13,
                (y2*N1-y1*N2)/F12 + (y2*N3- y3*N2)/F23])


res=solve_ivp(dy,(0,1),(y1i, y2i),method='Radau',dense_output=True)

In [ ]:
eta_plot=np.linspace(0,1,51)
sol=res.sol(eta_plot)

In [ ]:
fig=make_subplots(rows=1,cols=1)
fig.add_trace(go.Scatter(x=eta_plot*delta,y=sol[0],name='ethanol'))
fig.add_trace(go.Scatter(x=eta_plot*delta,y=sol[1],name='acetaldehye'))
fig.add_trace(go.Scatter(x=eta_plot*delta,y=1-sol[0]-sol[1],name='hydrogen'))